# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [ ]:
# Your solution goes here

In [1]:
# Install / import (usually already installed)
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

In [2]:
# convert rating text to number
rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}


In [3]:
#Scrape ONE page
base = "https://books.toscrape.com/"
url = base  # page 1

res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

books = soup.select("article.product_pod")

data = []

for b in books:
    title = b.h3.a["title"]
    price = b.select_one(".price_color").get_text(strip=True)        # e.g. "£51.77"
    availability = b.select_one(".availability").get_text(" ", strip=True)  # e.g. "In stock"
    
    rating_class = b.select_one("p.star-rating")["class"]            # ['star-rating', 'Three']
    rating_word = rating_class[1]
    rating = rating_map.get(rating_word, None)

    rel_link = b.h3.a["href"]
    product_url = urljoin(url, rel_link)

    data.append({
        "title": title,
        "price_gbp": price,
        "availability": availability,
        "rating": rating,
        "product_url": product_url
    })

df_books = pd.DataFrame(data)
df_books.head()


,title,price_gbp,availability,rating,product_url
0,A Light in the Attic,Â£51.77,In stock,3,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,Â£53.74,In stock,1,https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,Â£50.10,In stock,1,https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,Â£47.82,In stock,4,https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,5,https://books.toscrape.com/catalogue/sapiens-a...


In [4]:
#Scrape ALL pages
base = "https://books.toscrape.com/"
url = base  # start at page 1

all_rows = []

while url:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    books = soup.select("article.product_pod")

    for b in books:
        title = b.h3.a["title"]
        price = b.select_one(".price_color").get_text(strip=True)
        availability = b.select_one(".availability").get_text(" ", strip=True)

        rating_class = b.select_one("p.star-rating")["class"]
        rating_word = rating_class[1] if len(rating_class) > 1 else None
        rating = rating_map.get(rating_word, None)

        rel_link = b.h3.a["href"]
        product_url = urljoin(url, rel_link)

        all_rows.append({
            "title": title,
            "price_gbp": price,
            "availability": availability,
            "rating": rating,
            "product_url": product_url
        })

    # find "next" page link (if exists)
    next_btn = soup.select_one("li.next a")
    if next_btn:
        url = urljoin(url, next_btn["href"])
    else:
        url = None

df_books = pd.DataFrame(all_rows)
df_books.shape
df_books.head()


,title,price_gbp,availability,rating,product_url
0,A Light in the Attic,Â£51.77,In stock,3,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,Â£53.74,In stock,1,https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,Â£50.10,In stock,1,https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,Â£47.82,In stock,4,https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,5,https://books.toscrape.com/catalogue/sapiens-a...


In [6]:
#Clean the scraped data
df_books["price_gbp"] = (
    df_books["price_gbp"]
    .str.replace(r"[^\d.]", "", regex=True)
    .astype(float)
)

df_books["in_stock"] = df_books["availability"].str.contains("In stock", case=False)

df_books.describe()

,price_gbp,rating
count,1000.00000,1000.000000
mean,35.07035,2.923000
std,14.44669,1.434967
min,10.00000,1.000000
25%,22.10750,2.000000
50%,35.98000,3.000000
75%,47.45750,4.000000
max,59.99000,5.000000


In [7]:
df_books["price_gbp"].head()
df_books["price_gbp"].dtype

dtype('float64')

In [8]:
# Quick analysis
# top 10 most expensive books
df_books.sort_values("price_gbp", ascending=False).head(10)

# average price by rating
df_books.groupby("rating")["price_gbp"].mean().round(2)


rating
1    34.56
2    34.81
3    34.69
4    36.09
5    35.37
Name: price_gbp, dtype: float64

In [9]:
#Save to CSV
df_books.to_csv("books_to_scrape.csv", index=False)